### CONDA ENVIRONMENTS

For steps __1. preprocess__ and __2. mip__, `conda activate g5ht-pipeline`

For step __3. segment__, `conda activate segment-torch` or `conda activate torchcu129`

For step __4. spline, 5. orient, 6. warp, 7. reg__

In [1]:
import sys
import os
import importlib
import utils

# SPECIFY DATA TO PROCESS

In [2]:
# DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\fluorescent_beads_ch_align\20251219'
DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251224'

INPUT_ND2 = 'date-20251224_strain-ISg5HT-ADF-TeTx_condition-fedpatch_worm002.nd2'
# INPUT_ND2 = 'date-20251224_strain-ISg5HT-ADF-TeTx_condition-fedpatch_worm002_chan_alignment.nd2'

INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)

NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\noise\noise_042925.tif'

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 41

noise_stack = utils.get_noise_stack(NOISE_PTH, STACK_LENGTH)

num_frames, height, width, num_channels = utils.get_range_from_nd2(INPUT_ND2_PTH, stack_length=STACK_LENGTH) 
beads_alignment_file = utils.get_beads_alignment_file(INPUT_ND2_PTH)

print(INPUT_ND2)
print('Num z-slices: ', STACK_LENGTH)
print('Number of frames: ', num_frames)
print('Height: ', height)
print('width: ', width)
print('Number of channels: ', num_channels)
print('Beads alignment file: ', beads_alignment_file)

date-20251224_strain-ISg5HT-ADF-TeTx_condition-fedpatch_worm002.nd2
Num z-slices:  41
Number of frames:  1200
Height:  512
width:  512
Number of channels:  2
Beads alignment file:  C:\Users\munib\POSTDOC\DATA\g5ht-free\20251224\date-20251224_strain-ISg5HT-ADF-TeTx_condition-fedpatch_worm002_chan_alignment.nd2


# 0. PROCESS BEADS ALIGNMENT DATA (OPTIONAL, CHANGING THIS SO BEADS ARE PROCESSED SEAMLESSLY IN PIPELINE)

The registration parameters between green and red channels will be applied to worm recordings

### SHEAR CORRECT AND CHANNEL REGISTER

In [8]:
from preprocess_parallel import main as preprocess_nd2_parallel
_ = importlib.reload(sys.modules['preprocess_parallel'])

num_frames_beads, _, _, _ = utils.get_range_from_nd2(beads_alignment_file, stack_length=STACK_LENGTH) 

# # command-line arguments
sys.argv = ["", beads_alignment_file, "0", str(num_frames_beads-1), NOISE_PTH, STACK_LENGTH, 5, num_frames_beads, height, width, num_channels]

# # Call the main function
preprocess_nd2_parallel()

Processing 50 stacks (0-49) using 5 workers...


100%|██████████| 50/50 [03:27<00:00,  4.16s/it]


Parallel preprocessing complete.


### MIP

This step saved the median channel registration parameters, need to do this somewhere else

In [9]:
from mip import main as mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
sys.argv = ["", beads_alignment_file, STACK_LENGTH, num_frames_beads, 2]

# Call the main function
mip()

<tifffile.TiffWriter 'mip.tif'> writing nonconformant BigTIFF ImageJ


0000.tif written to mip.tif
0001.tif written to mip.tif
0002.tif written to mip.tif
0003.tif written to mip.tif
0004.tif written to mip.tif
0005.tif written to mip.tif
0006.tif written to mip.tif
0007.tif written to mip.tif
0008.tif written to mip.tif
0009.tif written to mip.tif
0010.tif written to mip.tif
0011.tif written to mip.tif
0012.tif written to mip.tif
0013.tif written to mip.tif
0014.tif written to mip.tif
0015.tif written to mip.tif
0016.tif written to mip.tif
0017.tif written to mip.tif
0018.tif written to mip.tif
0019.tif written to mip.tif
0020.tif written to mip.tif
0021.tif written to mip.tif
0022.tif written to mip.tif
0023.tif written to mip.tif
0024.tif written to mip.tif
0025.tif written to mip.tif
0026.tif written to mip.tif
0027.tif written to mip.tif
0028.tif written to mip.tif
0029.tif written to mip.tif
0030.tif written to mip.tif
0031.tif written to mip.tif
0032.tif written to mip.tif
0033.tif written to mip.tif
0034.tif written to mip.tif
0035.tif written to 

# 1. SHEAR CORRECTION

- shear corrects each volume
  - depending on each exposure time, it can take roughly half a second between the first and last frames of a volume, so any movements need to be corrected for
- creates one `.tif` for each volume and stores it in the `shear_corrected` directory

In [3]:
import shear_correct
_ = importlib.reload(sys.modules['shear_correct'])

# sys.argv = ["", nd2 file, start_frame, end_frame, noise_pth, stack_length, n_workers, num_frames, height, width, num_channels]
sys.argv = ["", INPUT_ND2_PTH, "0", str(num_frames-1), NOISE_PTH, STACK_LENGTH, 5, num_frames, height, width, num_channels]

# Call the main function
shear_correct.main()

Processing 1200 stacks (0-1199) using 5 workers...


 52%|█████▏    | 620/1200 [37:21<34:56,  3.62s/it]  


KeyboardInterrupt: 

# 2. CHANNEL ALIGNMENT

### 2a. GET MEDIAN CHANNEL ALIGNMENT PARAMETERS FROM ALL FRAMES

- If channel alignment file found, uses that, if not uses worm recording

In [4]:
import get_channel_alignment
_ = importlib.reload(sys.modules['get_channel_alignment'])

## set beads_alignment_file to None to use worm recording for channel alignment, even if beads file exists
# beads_alignment_file = None

if beads_alignment_file is not None:
    align_with_beads = True
    num_frames_beads, _, _, _ = utils.get_range_from_nd2(beads_alignment_file, stack_length=STACK_LENGTH) 
    sys.argv = ["", beads_alignment_file, "0", str(num_frames_beads-1), NOISE_PTH, STACK_LENGTH, 5, num_frames_beads, height, width, num_channels, align_with_beads]
else:
    align_with_beads = False
    sys.argv = ["", INPUT_ND2_PTH, "0", str(num_frames-1), NOISE_PTH, STACK_LENGTH, 5, num_frames, height, width, num_channels, align_with_beads]

# # Call the main function
get_channel_alignment.main()

Processing 50 stacks (0-49) using 5 workers...


  0%|          | 0/50 [00:01<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\munib\\POSTDOC\\DATA\\g5ht-free\\20251224\\date-20251224_strain-ISg5HT-ADF-TeTx_condition-fedpatch_worm002_chan_alignment_chan_alignment.nd2'

### 2b. APPLY MEDIAN CHANNEL ALIGNMENT PARAMETERS

In [ ]:
import apply_channel_alignment
_ = importlib.reload(sys.modules['apply_channel_alignment'])

# sys.argv = ["", nd2 file, start_frame, end_frame, noise_pth, stack_length, n_workers, num_frames, height, width, num_channels]
sys.argv = ["", INPUT_ND2_PTH, "0", str(num_frames-1), NOISE_PTH, STACK_LENGTH, 5, num_frames, height, width, num_channels]

# Call the main function
apply_channel_alignment.main()

# 1. PREPROCESS

### PARALLEL

if processing locally, would recommend setting the 6th arg (number of workers) to something less than 10 if you want to use your computer at the same time

TODO:

- make a new step 2, that is 'align channels'
- it should first look for a beads .nd2, if not falls back to previous method
- alternatively, in the case of not aligning with beads, preprocess should do the median parameters and apply them

In [ ]:
from preprocess_parallel import main as preprocess_nd2_parallel
_ = importlib.reload(sys.modules['preprocess_parallel'])

# command-line arguments
# sys.argv = ["", INPUT_ND2_PTH, "0", str(num_frames), NOISE_PTH, STACK_LENGTH, 5]

# if not None:
#     align_with_beads = True # set to true when processing a worm recording with beads that have already been processed
# else:
#     align_with_beads = False
align_with_beads = True
# sys.argv = ["", INPUT_ND2_PTH, "1068", str(num_frames-1), NOISE_PTH, STACK_LENGTH, 5, num_frames, height, width, num_channels, align_with_beads]
sys.argv = ["", INPUT_ND2_PTH, "1090", "1094", NOISE_PTH, STACK_LENGTH, 5, num_frames, height, width, num_channels, align_with_beads]

# Call the main function
preprocess_nd2_parallel()

Processing 5 stacks (1090-1094) using 5 workers...


100%|██████████| 5/5 [00:31<00:00,  6.35s/it]


Parallel preprocessing complete.


### SERIAL

In [3]:
from preprocess import main as preprocess_nd2
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
# sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH]
sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH, STACK_LENGTH, num_frames, height, width, num_channels]

# Call the main function
preprocess_nd2()

# 2. MAX INTENSITY PROJECTION

In [4]:
from mip import main as mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
sys.argv = ["", INPUT_ND2_PTH, STACK_LENGTH, num_frames, 10]

# Call the main function
mip()

<tifffile.TiffWriter 'mip.tif'> writing nonconformant BigTIFF ImageJ


0000.tif written to mip.tif
0001.tif written to mip.tif
0002.tif written to mip.tif
0003.tif written to mip.tif
0004.tif written to mip.tif
0005.tif written to mip.tif
0006.tif written to mip.tif
0007.tif written to mip.tif
0008.tif written to mip.tif
0009.tif written to mip.tif
0010.tif written to mip.tif
0011.tif written to mip.tif
0012.tif written to mip.tif
0013.tif written to mip.tif
0014.tif written to mip.tif
0015.tif written to mip.tif
0016.tif written to mip.tif
0017.tif written to mip.tif
0018.tif written to mip.tif
0019.tif written to mip.tif
0020.tif written to mip.tif
0021.tif written to mip.tif
0022.tif written to mip.tif
0023.tif written to mip.tif
0024.tif written to mip.tif
0025.tif written to mip.tif
0026.tif written to mip.tif
0027.tif written to mip.tif
0028.tif written to mip.tif
0029.tif written to mip.tif
0030.tif written to mip.tif
0031.tif written to mip.tif
0032.tif written to mip.tif
0033.tif written to mip.tif
0034.tif written to mip.tif
0035.tif written to 

# 3. SEGMENT

### on home pc
`conda activate segment-torch`

Uses a separate conda environment from the rest of the pipeline. create it using:
`conda env create -f segment_torch.yml`

### on lab pc
`conda activate torchcu129`

Uses a separate conda environment from the rest of the pipeline. create it following steps in:
`segment_torch_cu129_environment.yml`

### setup each time model weights change
Need to set path to model weights as `CHECKPOINT` in `eval_torch.py`

In [ ]:

import sys
import os

DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001_aligned'
INPUT_ND2 = 'date-20251121_strain-ISg5HT_condition-fedpatch_worm002.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)
OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]
MIP_PTH = os.path.join(DATA_PTH, 'mip.tif')

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 15

print(INPUT_ND2)


In [ ]:
from segment.segment_torch import main as segment_mip


# command-line arguments
sys.argv = ["", MIP_PTH]

segment_mip()


# 4. SPLINE

`conda activate g5ht-pipeline`

In [ ]:
import sys
import os
from spline import main as get_spline

# PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001\label.tif'
PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121\date-20251121_strain-ISg5HT_condition-fedpatch_worm002\label.tif'

# command-line arguments
sys.argv = ["", PTH]

get_spline()

# 5. ORIENT

`conda activate g5ht-pipeline`

In [ ]:
import sys
import os
from orient import main as find_orientation

# PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001\spline.json'
PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121\date-20251121_strain-ISg5HT_condition-fedpatch_worm002\spline.json'

# command-line arguments
sys.argv = ["", PTH, 108, 42]

find_orientation()

# 6. WARP

`conda activate g5ht-pipeline`

TODO: parallelize

In [ ]:
import sys
import os
from warp import main as warp_worm
from tqdm import tqdm
from utils import get_range_from_nd2


DATA_PTH = r"C:\\Users\\munib\\POSTDOC\\DATA\\g5ht-free\\20251028\\"
INPUT_ND2 = 'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001_aligned.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)
STACK_LENGTH = 41
num_frames, height, width, num_channels = get_range_from_nd2(INPUT_ND2_PTH, stack_length=STACK_LENGTH) 

# PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251121\date-20251121_strain-ISg5HT_condition-fedpatch_worm002'
PTH = os.path.join(DATA_PTH,os.path.splitext(INPUT_ND2)[0])

for i in tqdm(range(num_frames)):
    # command-line arguments
    sys.argv = ["", PTH, i]

    warp_worm()

### TEST PARALLEL (TODO)

In [ ]:
from warp_parallel import main as warp_worm_parallel

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'

# command-line arguments
sys.argv = ["", PTH, "0", "1199"]

# Call the main function
warp_worm_parallel()

# 7. REG

`conda activate g5ht-pipeline`

TODO: parallelize

In [ ]:
import sys
import os
from tqdm import tqdm
import importlib

from reg import main as reg_worm


PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001_aligned'

for i in tqdm(range(1200)):
    # command-line arguments
    sys.argv = ["", PTH, i, "1"]
    reg_worm()

### REGISTER WITH GFP+1 TO RFP

TRIM LAST RFP ZSLICE, TRIM FIRST GFP ZSLICE

seems to be that as of 20251204, all recordings were taken such that the i zslice in red channel corresponds to i+1 zslice in green channel

In [ ]:
import sys
import os
from tqdm import tqdm
import importlib

from reg_gfp_indexing import main as reg_worm

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001_aligned'

for i in tqdm(range(1200)):
    # command-line arguments
    sys.argv = ["", PTH, i, "1"]
    reg_worm()

### View fixed and move as one image (see reg_microfilm.ipynb)

# 8. QUANTIFY

`conda activate g5ht-pipeline`

Have to first label dorsal and ventral nerve rings and pharynx. See ...

In [ ]:
import sys
import os
from quantify import main as quantify_roi_intensity
from numpy import genfromtxt
import matplotlib.pyplot as plt
import numpy as np

import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 15}
matplotlib.rc('font', **font)

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001'


sys.argv = ["", PTH]
quantify_roi_intensity()

In [ ]:
roi_intensity = genfromtxt(os.path.join(PTH,'quantified.csv'), delimiter=',')
roi_intensity = roi_intensity[1:,:]
roi_intensity

In [ ]:
%matplotlib qt

t = roi_intensity[:,0]

plt.figure(figsize=(10, 4))
plt.plot(t, roi_intensity[:, 1] / np.mean(roi_intensity[:60, 1]), label='Dorsal nerve ring', lw=3)
plt.plot(t, roi_intensity[:, 2] / np.mean(roi_intensity[:60, 2]), label='Ventral nerve ring', lw=3)
plt.plot(t, roi_intensity[:, 3] / np.mean(roi_intensity[:60, 3]), label='Pharynx', lw=3)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
plt.xlabel('Time (min)')
plt.ylabel(r'$F/F_{baseline}$')
plt.xlim(t[0],t[-1])
plt.axhline(1, ls='--', c='k', zorder=0)
plt.tight_layout()
plt.show()
